In [1]:
# Gymnasium & Minigrid imports
import gymnasium as gym  # Correct way to import Gymnasium
from minigrid.core.constants import COLOR_NAMES
from minigrid.core.constants import DIR_TO_VEC
from minigrid.core.grid import Grid
from minigrid.core.actions import Actions
from minigrid.core.mission import MissionSpace
from minigrid.core.world_object import Door, Goal, Key, Wall
from minigrid.manual_control import ManualControl
from minigrid.minigrid_env import MiniGridEnv



from gymnasium.utils.play import play


import numpy as np

import matplotlib.pyplot as plt
from dataclasses import dataclass, field
from typing import Optional, List, Dict, Tuple

from scipy.ndimage import binary_dilation
import pandas as pd


from scipy.sparse import csr_matrix
from scipy.sparse.linalg import matrix_power
from scipy.sparse import lil_matrix

In [49]:
import pickle

In [50]:
with open("mask.pkl", "wb") as f:
    pickle.dump(mask, f)

In [51]:
with open("reward_struct.pkl", "wb") as f:
    pickle.dump(reward_struct, f)

In [225]:
print(gym.__version__)

1.0.0


In [5]:
%matplotlib inline

In [6]:
%store -r museum_mask_03m

In [7]:
mask = museum_mask_03m

In [8]:
mask = np.flipud(mask)

In [8]:
# np.save("images/walkability_mask.npy", walkability_mask)

# Environment

In [2]:
class SimplestEnv(MiniGridEnv):
    def __init__(
        self, 
        agent_start_pos=(7, 21), 
        agent_start_dir=0, 
        max_steps=700,
        mask=None, 
        **kwargs,
    ):
        assert mask is not None, "You must provide a walkability mask"
        height, width = mask.shape
        self.mask = mask
        self.agent_start_pos = agent_start_pos
        self.agent_start_dir = agent_start_dir
        self.goal_pos = 

        mission_space = MissionSpace(mission_func=self._gen_mission)

        super().__init__(
            mission_space=mission_space,
            grid_size=max(width, height),
            max_steps=max_steps,
            **kwargs,
        )

        self.width = width
        self.height = height
        self.action_space = gym.spaces.Discrete(3)

    @staticmethod
    def _gen_mission():
        return "Museum"



    def _gen_grid(self, width, height):
        self.grid = Grid(self.width, self.height)

        for y in range(self.height):
            for x in range(self.width):
                if not self.mask[y, x]:
                    self.grid.set(x, y, Wall())
        #place agent
        
        if self.agent_start_pos is not None:
            self.agent_pos = self.agent_start_pos #check this
            self.agent_dir = self.agent_start_dir
        else:
            self.place_agent()
        print(f"[DEBUG] Placing agent at {self.agent_pos} facing {self.agent_dir}")
        
        # Do not call self.place_agent() → we already set it
        self.mission = "Museum"
        print(f"[DEBUG] Initialized grid {self.width}x{self.height}, agent at {self.agent_pos}")
        


SyntaxError: invalid syntax (713913533.py, line 15)

In [18]:
env = SimpleEnv(render_mode="human", mask = mask)
obs, _ = env.reset()

[DEBUG] mask.shape = (37, 69), self.width = 69, self.height = 37


In [12]:
test_env = SimplestEnv(
     # (x, y) format → left edge, near bottom           # Facing up (0 = up, 1 = right, 2 = down, 3 = left)
    render_mode="human",
    mask=mask
)
test_env.reset(); 

[DEBUG] Placing agent at (7, 21) facing 0
[DEBUG] Initialized grid 69x37, agent at (7, 21)


# define reward structure

In [9]:
%store -r reward_grid_03  #import from computed notebook

no stored variable or alias #import
no stored variable or alias from
no stored variable or alias computed
no stored variable or alias notebook


In [10]:
reward_grid_03

array([[-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       ...,
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1]], shape=(37, 69))

In [11]:
#flip
reward_struct = np.flipud(reward_grid_03)

In [12]:
reward_struct

array([[-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       ...,
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1]], shape=(37, 69))

In [16]:

class SimpleEnv(MiniGridEnv):
    def __init__(
            self, 
            agent_start_pos=(1, 30), # bottom left entrance ( 1, 30) top right corner (63, 5)
            agent_start_dir=0, 
            max_steps=50,
            mask = None, 
            reward_mask = None,
            
            **kwargs,
    ):
        
    
        height, width = mask.shape
        self.agent_start_pos = agent_start_pos
        self.agent_start_dir = agent_start_dir
        self.agent_dir = 0 #facing up always
        # self.goal_pos = (30, 15)
        self.mask = mask
        self.reward_mask = reward_mask
        self.visited_reward_0 = set()  # holds (x, y) tuples

        
        
        
        mission_space = MissionSpace(mission_func=self._gen_mission)

        super().__init__(
            mission_space=mission_space,
            # grid_size=max(width, height),
            width=width,
            height=height,
            max_steps=max_steps,
            **kwargs,
        )

        # Restore correct values
        self.width = width
        self.height = height

        self.action_space = gym.spaces.Discrete(5)
    @staticmethod
    def _gen_mission():
        return "Museum"

    def _gen_grid(self, width, height):

        self.grid = Grid(self.width, self.height)

        for y in range(self.mask.shape[0]):
            for x in range(self.mask.shape[1]):
                if not self.mask[y, x]:
                    self.grid.set(x, y, Wall())



        #place goal
        # self.put_obj(Goal(), 30, 15)

        if self.agent_start_pos is not None:
            self.agent_pos = self.agent_start_pos #check this
            self.agent_dir = self.agent_start_dir
        else:
            self.place_agent()

        self.mission = "Museum"

        # print(f"agent_start = {self.agent_start_pos}")
        print(f"[DEBUG] Initialized grid {self.width}x{self.height}, agent at {self.agent_pos}")

    def step(self, action):
        x, y = map(int, self.agent_pos)

        # Define absolute movement
        if action == 0:      # stay
            dx, dy = 0, 0
        elif action == 1:    # up
            dx, dy = 0, -1
        elif action == 2:    # down
            dx, dy = 0, 1
        elif action == 3:    # left
            dx, dy = -1, 0
        elif action == 4:    # right
            dx, dy = 1, 0
        else:
            raise ValueError(f"Invalid action: {action}")

        new_x, new_y = int(x + dx), int(y + dy)

        # Stay in bounds
        if not (0 <= new_x < self.width and 0 <= new_y < self.height):
            new_x, new_y = x, y

        # Check for wall
        target_cell = self.grid.get(new_x, new_y)
        if target_cell is not None and not target_cell.can_overlap():
            new_x, new_y = x, y  # can't move into wall

        self.agent_pos = (new_x, new_y)

        # --- REWARD LOGIC ---
        if self.reward_mask[new_y, new_x] == 0:
            if (new_x, new_y) in self.visited_reward_0:
                reward = -1  # revisited exhibit
            else:
                reward = 0   # first time visit
                self.visited_reward_0.add((new_x, new_y))
        else:
            reward = self.reward_mask[new_y, new_x]


        obs = self.gen_obs()
        done = False
        info = {}

        return obs, reward, done, info
    
    
    def count_states(self):
        free_cells = sum(1 for x in range(self.grid.width)
                      for y in range(self.grid.height)
                      if not self.grid.get(x, y)) * 4
        return free_cells 


In [17]:

env = SimpleEnv(render_mode="human", mask = mask, reward_mask= reward_struct)
env.reset();

[DEBUG] Initialized grid 69x37, agent at (1, 30)


In [19]:
def position_to_state_index(pos=None):
    """Map (x, y) to state index assuming row-major order."""
    if pos is None:
        x, y = env.agent_pos
    else:
        x, y = pos
    return y * env.width + x

def state_index_to_position(idx):
    """Map scalar state index to (x, y) position."""
    x = idx % env.width
    y = idx // env.width
    return x, y

def find_state_indexes():
    """Return all non-wall, walkable state indices."""
    state_indices = []
    for y in range(env.height):
        for x in range(env.width):
            if env.grid.get(x, y) is None:  # Empty/walkable
                state_indices.append(position_to_state_index((x, y)))
    return state_indices

allowed_state_idx = find_state_indexes()

num_actions = 5
num_states = (env.width) * (env.height)


def next_state_index(current_state_idx, action):
    """Given a state index and action, return the resulting state index."""
    x, y = state_index_to_position(current_state_idx)

    # Movement logic
    if action == 0:  # stay
        pass
    elif action == 1:  # up
        y -= 1
    elif action == 2:  # down
        y += 1
    elif action == 3:  # left
        x -= 1
    elif action == 4:  # right
        x += 1
    else:
        raise ValueError(f"Invalid action: {action}")

    
    # Convert the new (x, y, direction) back to a state index.
    next_state = position_to_state_index((x, y))
    if next_state not in allowed_state_idx:
        # If the next state is not allowed, return the current state
        return current_state_idx
    return next_state
P_s_given_s_a = np.zeros((num_states, num_actions, num_states))

def find_all_next_states():
    """Find all possible next states for each state and action."""
    for state in allowed_state_idx:
        for action in range(num_actions):
            next_state = next_state_index(state, action)
            P_s_given_s_a[state, action, next_state] = 1 





In [43]:
obs, _ = env.reset()

for i in range(2000):
    action = env.action_space.sample()  # random action
        
    obs, reward, terminated, truncated, info = env.step(action)  # 5-tuple

    # Current state index from current (x,y,dir)
    idx = position_to_state_index()
    x, y, d = state_index_to_position(idx)
    idx = position_to_state_index()
    print(position_to_state_index())
    print(state_index_to_position(idx))
    
    
    print(f"Step {i}: action={action}, pos={env.agent_pos}, reward={reward}")
    env.render()  # <- show movement

[DEBUG] Initialized grid 69x37, agent at (1, 30)


ValueError: not enough values to unpack (expected 5, got 4)

: 

# Model State Class

In [21]:
@dataclass

class ModelState:
    #tables and arrays
    Q_table: np.ndarray = field(default_factory=lambda: np.array([]))
    Pi_a_s: np.ndarray = field(default_factory=lambda: np.array([]))
    # P_s_given_s_a: np.ndarray = field(default_factory=lambda: np.array([]))
    P_s_by_s: np.ndarray = field(default_factory=lambda: np.array([]))
    
    allowed_state_idx: np.ndarray = field(default_factory=lambda: np.array([]))
    
    #scalars
    beta: Optional[float] = None
    num_actions: Optional[int] = None
    num_states: Optional[int] = None

    #lists
    info_to_go_term_training: List[float] = field(default_factory=list)
    pi_analysis_term_training: List[float] = field(default_factory=list)

    #list for P_s
    positions_directions_list_ps: List[float] = field(default_factory=list)
    positions_directions_list_neglogps: List[float] = field(default_factory=list)

    #dictionaries
    transition_counts: Dict = field(default_factory=dict)
    connect_dict: Dict = field(default_factory=dict)

In [22]:
#instantiate state instance, can also import from another file if needed
state = ModelState()

# utility functions

In [23]:
class GridMixin:
    def position_to_state_index(self, pos=None):
        """Map (x, y) to state index assuming row-major order."""
        if pos is None:
            x, y = self.env.agent_pos
        else:
            x, y = pos
        return y * self.env.width + x

    def state_index_to_position(self, idx):
        """Map scalar state index to (x, y) position."""
        x = idx % self.env.width
        y = idx // self.env.width
        return x, y

    def find_state_indexes(self):
        """Return all non-wall, walkable state indices."""
        state_indices = []
        for y in range(self.env.height):
            for x in range(self.env.width):
                if self.env.grid.get(x, y) is None:  # Empty/walkable
                    state_indices.append(self.position_to_state_index((x, y)))
        return state_indices

    def next_state_index(self, current_state_idx, action):
        """Given a state index and action, return the resulting state index."""
        x, y = self.state_index_to_position(current_state_idx)

        # Movement logic
        if action == 0:  # stay
            pass
        elif action == 1:  # up
            y -= 1
        elif action == 2:  # down
            y += 1
        elif action == 3:  # left
            x -= 1
        elif action == 4:  # right
            x += 1
        else:
            raise ValueError(f"Invalid action: {action}")

        
        # Convert the new (x, y, direction) back to a state index.
        next_state = self.position_to_state_index((x, y))
        if next_state not in self.state.allowed_state_idx:
            # If the next state is not allowed, return the current state
            return current_state_idx
        return next_state
    
    
    def find_all_next_states(self):
        """Find all possible next states for each state and action."""
        self.state.connect_dict = {}
        for state in self.state.allowed_state_idx:
            tmp = []
            for action in range(self.state.num_actions):
                next_state = self.next_state_index(state, action)
                tmp.append((next_state, action))
            self.state.connect_dict[state] = tmp

            

    def find_connected_states(self):
        """
        Build and return a connectivity matrix P_s_by_s where the element at [state, next_state]
        is given by the probability from self.Pi_a_s for the action that leads from state to next_state.
        """
        # Reset the connectivity matrix at the beginning.
        # self.state.P_s_by_s = np.zeros((self.state.num_states, self.state.num_states))
        self.state.P_s_by_s = np.zeros((self.state.num_states, self.state.num_states))
        # Loop over all states.
        for state in self.state.allowed_state_idx:
            # Loop over all actions for the state.
            for pair in self.state.connect_dict[state]:
                s_n, action = pair
                # Set the connectivity matrix: you might choose to sum if multiple actions lead to the same state.
                self.state.P_s_by_s[state, s_n] += self.state.Pi_a_s[state, action]
    
    def update_connected_states(self, s):
        """Update the connectivity matrix P_s_by_s for a single state s."""
        for pair in self.state.connect_dict[s]:
                s_n, action = pair
                # Set the connectivity matrix: you might choose to sum if multiple actions lead to the same state.
                self.state.P_s_by_s[s, s_n] = self.state.Pi_a_s[s, action]

    
    def propagate_distribution(self, p0, A, steps):
        """
        Propagate initial distribution p0 through transition matrix A
        for given number of steps.
        A is CSR sparse matrix, row-stochastic.
        """
        if steps == 0:
            return p0.copy()
        else:
            p = p0.copy()
            for _ in range(steps):
                p = p @ A  # vector * sparse matrix
            return p

        


# Training

In [38]:
##################working code! ######################
class FreeEnergy(GridMixin):
    def __init__(self, env, state: ModelState, epochs = 200, beta = 0.16, goal_state = None):
        self.state = state
        self.env = env
        state: ModelState
        self.goal_state = goal_state   # Default goal state if not provided
        self.reward_mask = env.reward_mask
 
        # self.max_steps = 100
        self.beta = beta
        self.epochs = epochs
        self.state = state
        self.state.allowed_state_idx = self.find_state_indexes()
        self.state.num_states = env.width * env.height
        print(f"Number of states: {self.state.num_states}")
        self.state.num_actions = 5
        #shapes defined for first time
        # self.state.P_s_given_s_a = np.zeros((self.state.num_states, self.state.num_actions, self.state.num_states)).astype(int) # P(s'|s,a) matrix
        self.state.P_s_by_s = np.zeros((self.state.num_states, self.state.num_states)) # P(s'|s) matrix
        self.state.Pi_a_s = np.full((self.state.num_states, self.state.num_actions), 1/self.state.num_actions) # pi(s,a) matrix
      
        self.Free_energy_table = np.full((self.state.num_states, self.state.num_actions), 0.0) # Free energy table


        
        self.state.beta = beta

        
        self.Pi_a = np.full((self.state.num_actions), 1/self.state.num_actions)
        self.P_s = np.zeros(self.state.num_states)
        
       
    """ Perform training using BA updates on the Probabilistic Q Learning System """


    def train_BA(self):
        max_ba_iters = 10 #max iterations for the Blahut-Arimoto update
        tol = 1e-2  #tolerance for convergence
     #tolerance for convergence
        self.find_all_next_states() #generates self.P(s'|s,a) matrix
        
        self.find_connected_states() #P_s fixed by old Pi_a_s

        
        goal_states = [self.position_to_state_index((self.goal_state))]
        

        for epoch in range(self.epochs):
            self.steps = 0
            #self.env.reset()[0] #can reset the environment to start from the beginning
            current_state = np.random.choice(self.state.allowed_state_idx) #self.position_to_state_index()
            x,y = self.state_index_to_position(current_state)
            self.env.agent_pos = (x,y)
            # self.env.agent_dir = dir

            initial_s = np.zeros(self.state.num_states)
            initial_s[current_state] = 1

            self.visited_reward_0 = set()

            while current_state not in goal_states:  # and self.steps < self.max_steps:
                #get the action using pi
                action = np.random.choice(np.arange(self.state.num_actions), p = self.state.Pi_a_s[current_state])
                #transition to the next state
                # switched from 5 to 4 
                next_obs, _, done, _ = self.env.step(action)

                # print(f"next_observation: {next_obs}")
                next_state = self.position_to_state_index()

                if self.goal_state is None:
                    reward = 0 if next_state in goal_states else -1
                else:
                    x, y = self.state_index_to_position(next_state)  # or self.env.state_index_to_position
                    if self.reward_mask[y, x] == 0:
                        if (x, y) in self.visited_reward_0:
                            reward = -1
                        else:
                            reward = 0
                            self.visited_reward_0.add((x, y))
                    else:
                        reward = -1
             

                sparse_s_by_s = csr_matrix(self.state.P_s_by_s)
                P_s = self.propagate_distribution(p0 = initial_s, A = sparse_s_by_s, steps = self.steps)

                # Ps_s_matrix = matrix_power(self.state.P_s_by_s, self.steps)  # Convert dia → csr
                # P_s = initial_s @ Ps_s_matrix
                
                # P_s /= P_s.sum() #normalise
           
                
                assert np.isclose(np.sum(P_s), 1, atol= 1e-8), f"Sum of P(s) is not 1: {np.sum(P_s)}"
                
                #calculate the Free energy
                F_0 = (-np.log(P_s[next_state]+ 1e-15)) - (self.beta * reward)
                # print(f" P_s[current_state]: {P_s[current_state]}")
                # print (f"P_s[next_state]: {P_s[next_state]}")
            
                G_0 = np.sum((np.log(self.state.Pi_a_s[next_state] + 1e-15) - np.log(self.Pi_a+1e-15) + self.Free_energy_table[next_state]) * self.state.Pi_a_s[next_state])

                F_0 += G_0 

                self.Free_energy_table[current_state, action] = F_0 #np.min([F_0, 200]) #    #np.min([F_0, 50])
                #self.Free_energy_table[self.state.allowed_state_idx] = self.Free_energy_table[self.state.allowed_state_idx] - np.mean(self.Free_energy_table[self.state.allowed_state_idx])
                # print(f"F_0:{F_0}")
                    # self.Free_energy_table[self.state.allowed_state_idx] = self.Free_energy_table[self.state.allowed_state_idx] - np.mean(self.Free_energy_table[self.state.allowed_state_idx])
                
                

                #######loop of calculations#####
                for iteration in range(max_ba_iters):
                  

                    self.Pi_a = P_s@self.state.Pi_a_s 
                
                  
                    
                    assert np.isclose(np.sum(self.Pi_a), 1), f"Sum of Pi_a is not 1: {np.sum(self.Pi_a)}"

                    
                    # 1) log π(a)  — shape (A,)
                    log_pi_a = np.log(self.Pi_a + 1e-15)

                    # 2) log-joint   log π(a) − β F(s,a)  — broadcast to shape (S,A)
                    log_joint = log_pi_a  -self.Free_energy_table

                    # 3) row-shift: subtract max in each state to keep exp() ≤ 1
                    log_joint -= log_joint.max(axis=1, keepdims=True)

                    # 4) exponentiate and normalise each row
                    new_Pi_a_s = np.exp(log_joint)
                    new_Pi_a_s /= new_Pi_a_s.sum(axis=1, keepdims=True)   # Σ_a π(a|s)=1
                                       
                    #calculate Z, works only for small beta
                    # element_wise_a_by_q_table = self.Pi_a * np.exp(-  self.Free_energy_table)
                    
                    # zeta = np.sum((element_wise_a_by_q_table), axis = 1)
                   
                    
                    # #calculate for policy using partition fuction

                    # new_Pi_a_s = (self.Pi_a * np.exp(-  self.Free_energy_table))/ zeta.reshape(-1,1) 
                    diff = np.linalg.norm(new_Pi_a_s - self.state.Pi_a_s, ord='fro')
                    # print(f"Epoch {epoch}: Frobenius norm difference: {diff:.4f}") # with percent
                    
                    self.state.Pi_a_s = new_Pi_a_s
                    iteration += 1
                    if diff < tol:
                        # print(f"Convergence reached at epoch {epoch} "f"after {iteration} BA iteration(s); "f"Δ = {diff:.4e}")
                        break

                      # Frobenius norm for matrix difference
                # self.state.P_s_by_s = P_s_csr.tolil()
                self.update_connected_states(current_state)

                # #update state if only single goal state 
                if self.goal_state is not None and next_state in goal_states:
                    # print(f"Goal reached at state {next_state} in epoch {epoch} after {self.steps} steps.")
                    break
                # assert(False)
                current_state = next_state
                # print(f"Epoch {epoch}: Step {self.steps}, Current state: {current_state}")
                self.steps += 1
    
    

In [25]:
    
##################working code! ######################
class FreeEnergy(GridMixin):
    def __init__(self, env, state: ModelState, epochs = 20, beta = 5):
        self.state = state
        self.env = env
        state: ModelState
         # self.max_steps = 100
        self.beta = beta
        self.epochs = epochs
        self.state = state
        self.state.allowed_state_idx = self.find_state_indexes()
        self.state.num_states = env.width * env.height
        self.state.num_actions = 5
        #shapes defined for first time
        # self.state.P_s_given_s_a = np.zeros((self.state.num_states, self.state.num_actions, self.state.num_states)) # P(s'|s,a) matrix
        self.state.P_s_by_s = np.zeros((self.state.num_states, self.state.num_states)) # P(s'|s) matrix
        self.state.Pi_a_s = np.full((self.state.num_states, self.state.num_actions), 1/self.state.num_actions) # pi(s,a) matrix
      
        self.Free_energy_table = np.full((self.state.num_states, self.state.num_actions), 0.0) # Free energy table


        self.reward_mask = reward_struct
        self.state.beta = beta

        
        self.Pi_a = np.full((self.state.num_actions), 1/self.state.num_actions)
        self.P_s = np.zeros(self.state.num_states)
        self.visited_reward_0 = set()  # holds (x, y) tuples

    def train_BA(self):
        max_ba_iters = 10
        tol = 1e-2

        goal_states = [self.position_to_state_index((30, 15))]  # Goal at (30, 15)
       
        

        self.find_all_next_states() #generates self.P(s'|s,a) matrix
        
        self.find_connected_states() #P_s fixed by old Pi_a_s
        
        for epoch in range(self.epochs):
            print(f"Epoch {epoch }/{self.epochs} starting...")
            self.steps = 0
            current_state = np.random.choice(self.state.allowed_state_idx)  #2071
            #print
            # print(f"current state: {current_state}")
            x, y = self.state_index_to_position(current_state)
            # print(f"current position: {x}, {y}")
            self.env.agent_pos = (x, y)
            # print(f"current agent position: {self.env.agent_pos}")
          

            initial_s = np.zeros(self.state.num_states)
            initial_s[current_state] = 1

            while self.steps < 150:  # ← Now limits to 100 steps per epoch
                action = np.random.choice(np.arange(self.state.num_actions), p=self.state.Pi_a_s[current_state])
                # print(f"Step {self.steps}: action={action}, current_state={current_state}")
                _, _, _, _ = self.env.step(action)
                # self.env.render()
                next_state = self.position_to_state_index()
                # print(f"next state: {next_state}")

                # reward = self.env.reward_mask[self.env.agent_pos[1], self.env.agent_pos[0]] \
                #         if self.env.reward_mask is not None else 0

                # if self.reward_mask[self.env.agent_pos[1], self.env.agent_pos[0]] == 0:
                #     if (self.env.agent_pos[0], self.env.agent_pos[1]) in self.visited_reward_0:
                #         reward = -1  # revisited exhibit
                #     else:
                #         reward = 0   # first time visit
                #         self.visited_reward_0.add((self.env.agent_pos[0], self.env.agent_pos[1]))
                # else:
                #     reward = self.reward_mask[self.env.agent_pos[1], self.env.agent_pos[0]]

                reward = 0 if next_state in goal_states else -1

                Ps_s_matrix = matrix_power(self.state.P_s_by_s, self.steps)  # Convert dia → csr
        
         
                P_s = initial_s @ Ps_s_matrix
                # P_s /= P_s.sum() #normalise
           
                
                assert np.isclose(np.sum(P_s), 1, atol= 1e-8), f"Sum of P(s) is not 1: {np.sum(P_s)}"

             
                F_0 = (-np.log(P_s[next_state]+ 1e-15)) - (self.beta * reward)
                # print(f" P_s[current_state]: {P_s[current_state]}")
                # print (f"P_s[next_state]: {P_s[next_state]}")
            
                G_0 = np.sum((np.log(self.state.Pi_a_s[next_state] + 1e-15) - np.log(self.Pi_a+1e-15) + self.Free_energy_table[next_state]) * self.state.Pi_a_s[next_state])

                F_0 += G_0 

                self.Free_energy_table[current_state, action] = F_0 #np.min([F_0, 200]) #    #np.min([F_0, 50])

                #######loop of calculations#####
                for iteration in range(max_ba_iters):
                  

                    self.Pi_a = P_s@self.state.Pi_a_s 
                
                  
                    
                    assert np.isclose(np.sum(self.Pi_a), 1), f"Sum of Pi_a is not 1: {np.sum(self.Pi_a)}"

                    
                    # 1) log π(a)  — shape (A,)
                    log_pi_a = np.log(self.Pi_a + 1e-15)

                    # 2) log-joint   log π(a) − β F(s,a)  — broadcast to shape (S,A)
                    log_joint = log_pi_a  -self.Free_energy_table

                    # 3) row-shift: subtract max in each state to keep exp() ≤ 1
                    log_joint -= log_joint.max(axis=1, keepdims=True)

                    # 4) exponentiate and normalise each row
                    new_Pi_a_s = np.exp(log_joint)
                    new_Pi_a_s /= new_Pi_a_s.sum(axis=1, keepdims=True)   # Σ_a π(a|s)=1
                                       
                    #calculate Z, works only for small beta
                    # element_wise_a_by_q_table = self.Pi_a * np.exp(-  self.Free_energy_table)
                    
                    # zeta = np.sum((element_wise_a_by_q_table), axis = 1)
                   
                    
                    # #calculate for policy using partition fuction

                    # new_Pi_a_s = (self.Pi_a * np.exp(-  self.Free_energy_table))/ zeta.reshape(-1,1) 
                    diff = np.linalg.norm(new_Pi_a_s - self.state.Pi_a_s, ord='fro')
                    # print(f"Epoch {epoch}: Frobenius norm difference: {diff:.4f}") # with percent
                    
                    self.state.Pi_a_s = new_Pi_a_s
                    iteration += 1
                    if diff < tol:
                        # print(f"Convergence reached at epoch {epoch} "f"after {iteration} BA iteration(s); "f"Δ = {diff:.4e}")
                        break

                      # Frobenius norm for matrix difference

                self.update_connected_states(current_state)

                 #update state
                if next_state in goal_states:
                    # print(f"Goal reached at state {next_state} in epoch {epoch} after {self.steps} steps.")
                    break

                # assert(False)
                current_state = next_state
                # print(f"Epoch {epoch}: Step {self.steps}, Current state: {current_state}")
                self.steps += 1
                # print(f"Epoch {epoch}: Step {self.steps}, Current state: {current_state}, Action: {action}, Reward: {reward}")

In [39]:
env = SimpleEnv(render_mode= None, mask = mask, reward_mask= reward_struct)
#env.reset needed to bypass the step counter function which is otherwise needed but doesn't exist in SimpleEnv
env.reset();

[DEBUG] Initialized grid 69x37, agent at (1, 30)


In [40]:
free = FreeEnergy(env, state, beta = 5)

In [41]:
free.train_BA()

AssertionError: Sum of P(s) is not 1: 0.9959207624190647

In [379]:
test_env = SimpleEnv(
    agent_start_pos= None,     # (x, y) format → left edge, near bottom           # Facing up (0 = up, 1 = right, 2 = down, 3 = left)
    render_mode="human",
    mask=mask,
    reward_mask=reward_struct
)
test_env.reset(); 

[DEBUG] Initialized grid 69x37, agent at (np.int64(7), np.int64(11))


In [364]:
state.Pi_a_s[1303]

array([1.63895302e-06, 1.01031555e-15, 6.43036042e-09, 9.99988369e-01,
       9.98522528e-06])

In [331]:
class QRunner(GridMixin):
    def __init__(self, env, state: ModelState):
        self.state = state
        self.env = env
        
    
    def run_policy_2(self):
        """Run the environment using the learned policy from a Q-table."""

        # self.env.reset()[0]  # Reset environment
        current_state = self.position_to_state_index()  # Convert starting position to index
        done = False
        self.step_count = 0  # Track steps to prevent infinite loops

        while not done and self.step_count < 1000:  # Prevent infinite loops
  
            action = np.random.choice(np.arange(self.state.num_actions), p=self.state.Pi_a_s[current_state])  # Choose best action
            next_obs, _, done, _ = self.env.step(action)  # Take action
            next_state = self.position_to_state_index()  # Convert new state
            self.env.render()  # Visualize movement
            # Calculate info to go term for the current step
            print(f"current state: {current_state}, action: {action}, next state: {next_state}")
            current_state = next_state  # Update current state
            self.step_count += 1

In [380]:
runner = QRunner(test_env, state)

In [213]:
state_index_to_position(951)

(54, 13)

In [381]:
runner.run_policy_2()

current state: 766, action: 0, next state: 766
current state: 766, action: 0, next state: 766
current state: 766, action: 0, next state: 766
current state: 766, action: 0, next state: 766
current state: 766, action: 0, next state: 766
current state: 766, action: 0, next state: 766
current state: 766, action: 0, next state: 766
current state: 766, action: 0, next state: 766
current state: 766, action: 0, next state: 766
current state: 766, action: 0, next state: 766
current state: 766, action: 0, next state: 766
current state: 766, action: 0, next state: 766
current state: 766, action: 0, next state: 766
current state: 766, action: 0, next state: 766
current state: 766, action: 0, next state: 766


KeyboardInterrupt: 

In [25]:
%store -r state_lookup_03

In [27]:
state_lookup = state_lookup_03

# testing